# SST: IPO & PDO

In [ ]:
import sys
sys.path.append("..")
import scipy as sp
import numpy as np
import xarray as xr
import seaborn as sns
import cmocean
import cartopy
import cartopy.crs as ccrs
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches

In [ ]:
%matplotlib inline
%config InlineBackend.print_figure_kwargs={'bbox_inches':None}
%load_ext autoreload
%autoreload 2
%aimport - numpy - scipy - matplotlib.pyplot

In [ ]:
from OHC import t2da, t2ds
from SST import SST_index, EOF_SST_analysis
from maps import map_robinson, map_eq_earth
from grid import find_array_idx
from paths import path_results, path_samoc, file_ex_ocn_ctrl, file_ex_ocn_rect
from regions import boolean_mask, SOM_area, global_ocean, gl_ocean_rect, NPacific_mask_rect, TPI_masks
from plotting import shifted_color_map, discrete_cmap
from timeseries import IterateOutputCESM, lowpass
from xr_DataArrays import xr_AREA
from xr_regression import xr_linear_trends_2D, xr_linear_trend, ocn_field_regression

# global yearly avg. SST map and trends

In [ ]:
MASK = xr.open_dataset(file_ex_ocn_ctrl, decode_times=False).REGION_MASK

In [ ]:
# %%time
# # ca. 4:30 min for ctrl/rcp, 1:25 for lpi
# # stacking files into one xr Dataset object
# for run in ['lpi']:  #['ctrl', 'rcp', 'lpd']:
#     for i, (y,m,s) in enumerate(IterateOutputCESM('ocn', run, 'yrly', name='TEMP_PD')):
#         print(y)
#         da = xr.open_dataset(s, decode_times=False).TEMP[0,:,:]
#         da = da.drop(['z_t', 'ULONG', 'ULAT'])
#         da['TLAT' ] = da['TLAT' ].round(decimals=2)
#         da['TLONG'] = da['TLONG'].round(decimals=2)
#         del da.encoding["contiguous"]
#         ds = t2ds(da=da, name='SST', t=int(round(da.time.item())))
#         ds.to_netcdf(path=f'{path_samoc}/SST/SST_yrly_{run}_{y}.nc', mode='w')
     
#     combined = xr.open_mfdataset(f'{path_samoc}/SST/SST_yrly_{run}_*.nc',
#                                  concat_dim='time',
#                                  autoclose=True,
#                                  coords='minimal')
#     combined.to_netcdf(f'{path_samoc}/SST/SST_yrly_{run}.nc')
#     # remove extra netCDF files

CPU times: user 1min 25s, sys: 3min 16s, total: 4min 41s
Wall time: 11min 18s

In [ ]:
SST_yrly_ctrl = xr.open_dataarray(f'{path_samoc}/SST/SST_yrly_ctrl.nc')
SST_yrly_rcp  = xr.open_dataarray(f'{path_samoc}/SST/SST_yrly_rcp.nc' )
SST_yrly_lpd  = xr.open_dataarray(f'{path_samoc}/SST/SST_yrly_lpd.nc' )
SST_yrly_lpi  = xr.open_dataarray(f'{path_samoc}/SST/SST_yrly_lpi.nc' )

In [ ]:
from paths import path_ocn_ctrl, path_ocn_lpd

In [ ]:
ds = xr.open_dataset(file_ex_ocn_ctrl, decode_times=False).TEMP

In [ ]:
ds

# SST indices

In [ ]:
MASK_ocn  = boolean_mask('ocn'     , 0)
MASK_low  = boolean_mask('ocn_low' , 0)
MASK_rect = boolean_mask('ocn_rect', 0)

In [ ]:
TAREA       = xr_AREA('ocn')
AREA_low    = xr_AREA('ocn_low')
AREA_rect   = xr_AREA('ocn_rect')

In [ ]:
AREA_low.plot()

In [ ]:
global_area = TAREA.where(MASK_ocn).sel(global_ocean).sum()
print(f'{global_area.item():e}')
global_area2 = AREA_rect.where(MASK_rect).sel(gl_ocean_rect).sum()
print(f'{global_area2.item():e}')

In [ ]:
from SST import SST_index_from_monthly

In [ ]:
%%time
TPI3_lpd = SST_index_from_monthly(run='lpd', index_loc=None, MASK=TPI_masks('ocn_low', 3))

# Interdecadal Pacific Oscillation

Henley at al. (2015): _A Tripole Index for the Interdecadal Pacific Oscillation_

In [ ]:
TPI1_area     = TAREA   .where(TPI_masks('ocn'    , 1)).sum()
TPI1_area_low = AREA_low.where(TPI_masks('ocn_low', 1)).sum()
TPI2_area     = TAREA   .where(TPI_masks('ocn'    , 2)).sum()
TPI2_area_low = AREA_low.where(TPI_masks('ocn_low', 2)).sum()
TPI3_area     = TAREA   .where(TPI_masks('ocn'    , 3)).sum()
TPI3_area_low = AREA_low.where(TPI_masks('ocn_low', 3)).sum()
print(TPI1_area, TPI1_area_low, TPI2_area, TPI2_area_low, TPI3_area, TPI3_area_low)

In [ ]:
%%time
# 3:15 min
dims = ('nlat', 'nlon')
TPI1_ctrl = SST_index(xa_SST=SST_yrly_ctrl, AREA=TAREA   , index_loc=None, AREA_index=TPI1_area    , MASK=TPI_masks('ocn'    , 1), dims=dims)
TPI1_rcp  = SST_index(xa_SST=SST_yrly_rcp , AREA=TAREA   , index_loc=None, AREA_index=TPI1_area    , MASK=TPI_masks('ocn'    , 1), dims=dims)
TPI1_lpd  = SST_index(xa_SST=SST_yrly_lpd , AREA=AREA_low, index_loc=None, AREA_index=TPI1_area_low, MASK=TPI_masks('ocn_low', 1), dims=dims)
TPI1_lpi  = SST_index(xa_SST=SST_yrly_lpi , AREA=AREA_low, index_loc=None, AREA_index=TPI1_area_low, MASK=TPI_masks('ocn_low', 1), dims=dims)

TPI2_ctrl = SST_index(xa_SST=SST_yrly_ctrl, AREA=TAREA   , index_loc=None, AREA_index=TPI2_area    , MASK=TPI_masks('ocn'    , 2), dims=dims)
TPI2_rcp  = SST_index(xa_SST=SST_yrly_rcp , AREA=TAREA   , index_loc=None, AREA_index=TPI2_area    , MASK=TPI_masks('ocn'    , 2), dims=dims)
TPI2_lpd  = SST_index(xa_SST=SST_yrly_lpd , AREA=AREA_low, index_loc=None, AREA_index=TPI2_area_low, MASK=TPI_masks('ocn_low', 2), dims=dims)
TPI2_lpi  = SST_index(xa_SST=SST_yrly_lpi , AREA=AREA_low, index_loc=None, AREA_index=TPI2_area_low, MASK=TPI_masks('ocn_low', 2), dims=dims)

TPI3_ctrl = SST_index(xa_SST=SST_yrly_ctrl, AREA=TAREA   , index_loc=None, AREA_index=TPI3_area    , MASK=TPI_masks('ocn'    , 3), dims=dims)
TPI3_rcp  = SST_index(xa_SST=SST_yrly_rcp , AREA=TAREA   , index_loc=None, AREA_index=TPI3_area    , MASK=TPI_masks('ocn'    , 3), dims=dims)
TPI3_lpd  = SST_index(xa_SST=SST_yrly_lpd , AREA=AREA_low, index_loc=None, AREA_index=TPI3_area_low, MASK=TPI_masks('ocn_low', 3), dims=dims)
TPI3_lpi  = SST_index(xa_SST=SST_yrly_lpi , AREA=AREA_low, index_loc=None, AREA_index=TPI3_area_low, MASK=TPI_masks('ocn_low', 3), dims=dims)

In [ ]:
TPI1_ctrl = TPI2_ctrl - (TPI1_ctrl+TPI3_ctrl)/2
TPI1_rcp  = TPI2_rcp  - (TPI1_rcp +TPI3_rcp )/2
TPI1_lpd  = TPI2_lpd  - (TPI1_lpd +TPI3_lpd )/2
TPI1_lpi  = TPI2_lpi  - (TPI1_lpi +TPI3_lpi )/2

In [ ]:
TPI1_ctrl.to_netcdf(f'{path_results}/SST/TPI_ctrl.nc')
TPI1_rcp .to_netcdf(f'{path_results}/SST/TPI_rcp.nc')
TPI1_lpd .to_netcdf(f'{path_results}/SST/TPI_lpd.nc')
TPI1_lpi .to_netcdf(f'{path_results}/SST/TPI_lpi.nc')

In [ ]:
TPI1_ctrl = xr.open_dataarray(f'{path_results}/SST/TPI_ctrl.nc')
TPI1_rcp  = xr.open_dataarray(f'{path_results}/SST/TPI_rcp.nc' )
TPI1_lpd  = xr.open_dataarray(f'{path_results}/SST/TPI_lpd.nc' )
TPI1_lpi  = xr.open_dataarray(f'{path_results}/SST/TPI_lpi.nc' )

In [ ]:
plt.plot(lowpass(TPI1_ctrl, 13))
plt.plot(lowpass(TPI1_rcp , 13))
plt.plot(lowpass(TPI1_lpd , 13))
plt.plot(lowpass(TPI1_lpi , 13))

# Pacific Decadal Oscillation

Mantua et al. (1997)

> The leading EOF of monthlySST anomalies over the North Pacific (after removing the global mean SST anomaly) and its associated PC time series are termed the Pacific Decadal Oscillation (PDO)

1. compute monthly global mean SST

2. create North Pacific monthly output fields
    2.1. determine extend of grid
    2.2. reduce all coordinates appropriately
    2.3. loop through all months and save as single netcdf

3. EOF analysis

maybe use ocn rect for it

In [ ]:
# %%time
# # 29 min
# for run in ['ctrl', 'rcp']:
#     for i, (y,m,s) in enumerate(IterateOutputCESM(domain='ocn', run=run, tavg='monthly')):
#         SST = xr.open_dataset(s, decode_times=False).TEMP[0,0,:,:]
#         SST_gm = SST_index(xa_SST=SST, AREA=TAREA, index_loc=global_ocean, AREA_index=global_area, MASK=MASK_ocn, dims=('nlat', 'nlon'))
#         if m==1: print(y, SST_gm.item()) 
#         if i==0:  SST_new = SST_gm
#         else:     SST_new = xr.concat([SST_new, SST_gm], dim='time')
#     SST_new.to_netcdf(f'{path_results}/SST_global_mean_monthly_{run}.nc')

In [ ]:
# %%time
# # 11 min
# for run in ['ctrl', 'rcp']:
#     for i, (y,m,s) in enumerate(IterateOutputCESM(domain='ocn_rect', run=run, tavg='monthly')):
#         SST = xr.open_dataset(s, decode_times=False).TEMP[0,:,:]
#         SST_gm = SST_index(xa_SST=SST, AREA=AREA_rect, index_loc=gl_ocean_rect, AREA_index=global_area2, MASK=MASK_rect, dims=('t_lat', 't_lon'))
#         if m==1: print(y, SST_gm.item()) 
#         if i==0:  SST_new = SST_gm
#         else:     SST_new = xr.concat([SST_new, SST_gm], dim='time')
#     SST_new.to_netcdf(f'{path_results}/SST_global_mean_monthly_rect_{run}.nc')

In [ ]:
SST_gm_ctrl      = xr.open_dataarray(f'{path_results}/SST_global_mean_monthly_ctrl.nc'     , decode_times=False)
SST_gm_rcp       = xr.open_dataarray(f'{path_results}/SST_global_mean_monthly_rcp.nc'      , decode_times=False)
SST_gm_rect_ctrl = xr.open_dataarray(f'{path_results}/SST_global_mean_monthly_rect_ctrl.nc', decode_times=False)
SST_gm_rect_rcp  = xr.open_dataarray(f'{path_results}/SST_global_mean_monthly_rect_rcp.nc' , decode_times=False)

In [ ]:
plt.plot(SST_gm_ctrl)
plt.plot(SST_gm_rcp )
plt.plot(SST_gm_rect_ctrl)
plt.plot(SST_gm_rect_rcp )

In [ ]:
plt.axhline(0, c='k', lw=.5)
plt.plot(SST_gm_rect_ctrl.time/12+100, SST_gm_rect_ctrl-SST_gm_ctrl[:len(SST_gm_rect_ctrl)])
plt.plot(SST_gm_rect_rcp .time/12+300, SST_gm_rect_rcp -SST_gm_rcp [:len(SST_gm_rect_rcp )])

somehow there is an almost constant low bias in the global mean SST when calculatd in the rect files

In [ ]:
NPac_MASK = NPacific_mask_rect()
NPac_area = xr_AREA('ocn_rect').where(NPac_MASK, drop=True)

In [ ]:
# %%time
# # 51 mins for 162 years of ctrl
# # 9 min for 67 years of rcp
# 
# for j, run in enumerate(['ctrl', 'rcp']):
#     SST_gm_rect = [SST_gm_rect_ctrl, SST_gm_rect_rcp][j]
#     for i, (y,m,s) in enumerate(IterateOutputCESM(domain='ocn_rect', tavg='monthly', run=run)):
#         if m==1:
#             print(y)
#         xa = xr.open_dataset(s, decode_times=False).TEMP[0,:,:].where(NPac_MASK, drop=True) - SST_gm_rect[i]
#         if i==0:
#             xa_out = xa.copy()
#         else:
#             xa_out = xr.concat([xa_out, xa], dim='time')
#         xa_out.to_netcdf(f'{path_samoc}/SST/NPacific_anomaly_monthly_rect_{run}.nc')

In [ ]:
SST_NPac_rect_ctrl = xr.open_dataarray(f'{path_samoc}/SST/NPacific_anomaly_monthly_rect_ctrl.nc', decode_times=False)
SST_NPac_rect_rcp  = xr.open_dataarray(f'{path_samoc}/SST/NPacific_anomaly_monthly_rect_rcp.nc' , decode_times=False)

In [ ]:
# %%time
# # 30 sec for rcp
# # 2 mins for ctrl
# fn = f'{path_results}/SST/NPac_ctrl.nc'
# eof_ctrl, pc_ctrl = EOF_SST_analysis(xa=SST_NPac_rect_ctrl, weights=NPac_area, fn=fn)
# fn = f'{path_results}/SST/NPac_rcp.nc'
# eof_rcp , pc_rcp  = EOF_SST_analysis(xa=SST_NPac_rect_rcp , weights=NPac_area, fn=fn)

In [ ]:
NPac_ctrl = xr.open_dataset(f'{path_results}/SST/NPac_ctrl.nc', decode_times=False)
NPac_rcp  = xr.open_dataset(f'{path_results}/SST/NPac_rcp.nc' , decode_times=False)

In [ ]:
plt.figure(figsize=(8,5))
plt.tick_params(labelsize=14)
plt.axhline(0, c='k', lw=.5)
plt.plot(NPac_ctrl.time/12+100, NPac_ctrl.pcs.rolling(time= 1, center=True).mean(), c='C0', lw=1)
plt.plot(NPac_rcp .time/12+200, NPac_rcp .pcs.rolling(time= 1, center=True).mean(), c='C1', lw=1)
plt.plot(NPac_ctrl.time/12+100, NPac_ctrl.pcs.rolling(time=12, center=True).mean(), c='C0', lw=3)
plt.plot(NPac_rcp .time/12+200, NPac_rcp .pcs.rolling(time=12, center=True).mean(), c='C1', lw=3)
plt.xlabel('time [years]'             , fontsize=16)
plt.ylabel('first principal component', fontsize=16)

In [ ]:
f, ax = plt.subplots(1,1,figsize=(8,5))
ax.tick_params(labelsize=14)
ax.axhline(0, c='k', lw=.5)
ax.plot(NPac_ctrl.time/12+100, NPac_ctrl.pcs.rolling(time= 12, center=True).mean(), c='C0', lw=1)
ax.plot(NPac_rcp .time/12+200, NPac_rcp .pcs.rolling(time= 12, center=True).mean(), c='C1', lw=1)
ax.plot(NPac_ctrl.time/12+100, NPac_ctrl.pcs.rolling(time=120, center=True).mean(), c='C0', lw=3)
ax.plot(NPac_rcp .time/12+200, NPac_rcp .pcs.rolling(time=120, center=True).mean(), c='C1', lw=3)
ax.set_xlabel('time [years]'             , fontsize=16)
ax.set_ylabel('PDO', fontsize=16)
ax.text(.02, .02, '10 year running mean', transform=ax.transAxes, fontsize=14, color='grey')
plt.tight_layout()
plt.savefig(f'{path_results}/SST/PDO_index')

In [ ]:
xa = NPac_ctrl.eofs[0]
cmap = discrete_cmap(14,  shifted_color_map(cmocean.cm.balance, start=.5-1./12, midpoint=0.5, stop=1., name='shrunk'))
f = plt.figure(figsize=(8,5))
ax = plt.axes(projection=ccrs.PlateCarree(central_longitude=190))
# ax.set_extent([.02,.98,.12,.86])
im1 = ax.pcolormesh(xa.t_lon, xa.t_lat, xa.values,
              cmap=cmap, vmin=-1, vmax=6,
                           transform=ccrs.PlateCarree() )
ax.add_feature(cartopy.feature.LAND, facecolor='grey', edgecolor='k', alpha=0.5)
cb = plt.colorbar(im1, orientation='horizontal')
cb.set_label('correlation coefficient', fontsize=12)
ax.set_title('EOF1 expressed as correlation', fontsize=16)
plt.savefig(f'{path_results}/SST/PDO_EOF', bbox_inches='tight')
# plt.tight_layout()
# (NPac_rcp.eofs-NPac_ctrl.eofs).plot()